# 5. Topic Labelling
In this notebook we will be infering a label for each one of the topics obtained in the Topic Modelling notebook. The main process will be divided as follows:
* We will begin by linking to [Wikidata](https://www.wikidata.org/wiki/Wikidata:Main_Page) each one of the terms that belong to each topic. (_Entity linking_)
* After the entity linking step, we will generate a Graph with the neighbour nodes of each starting term by traversing their Wikidata information based on a set of properties to expand.
* Once a graph is built for each topic, we will remove isolated subgraphs from them to obtain the main connected subgraph of each one.
* We will apply a set of centrality algorithms to obtain the entity from Wikidata which best represents the topic subgraph. This node will be used as a label for the topic.
* Finally, the topic labels will be added to the final model, which will be serialized for further use in the following notebooks.

## Setup
As always, we will begin by loading a set of constants and initializing the logging system. Since we will be using Bokeh in this notebook, we will configure it to output the results in the Jupyter notebook:

In [1]:
%run __init__.py

In [2]:
from bokeh.io import output_notebook

output_notebook()

/home/alejandro/.envs/edma/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


Loading BokehJS ...

## Entity linking


### Defining the entity linking class

In [3]:
import json
import requests


WIKIDATA_BASE = "https://www.wikidata.org/w"

class WikidataEntityLinker():
    def fit(self, X, y, *args):
        return self
    
    def transform(self, X, y, *args):
        return [self.link_entity(entity) 
                for doc in X
                for entity in doc]
    
    def link_entity(self, entity_label):
        url = f"{WIKIDATA_BASE}/api.php?action=wbsearchentities&search=" + \
            f"{entity_label}&language=en&format=json"
        response = requests.get(url)
        if response.status_code != 200:
            raise Error()
        content = json.loads(response.text)
        search_results = content['search']
        if len(search_results) == 0:
            return (entity_label, None)
        return (entity_label, search_results[0]['concepturi'])


In [4]:
entity_linker = WikidataEntityLinker()
res = entity_linker.link_entity('agroforestry')
res

('agroforestry', 'http://www.wikidata.org/entity/Q397350')

### Linking each topic's term to Wikidata

In [6]:
from src.utils import load_object

lda_agriculture_pipe_filename = "agriculture_lda_model.pkl"
dtm_tf_filename = "agriculture_dtm_tf.pkl"

lda_pipe = load_object(os.path.join(NOTEBOOK_3_RESULTS_DIR, lda_agriculture_pipe_filename))
dtm_tf = load_object(os.path.join(NOTEBOOK_3_RESULTS_DIR, dtm_tf_filename))

In [7]:
from src.utils import get_topic_terms_by_relevance

def link_topic_terms(entity_linker, model, vectorizer,
                     dtm_tf, n_top_words, lambda_=0.6):
    res = []
    topic_terms = get_topic_terms_by_relevance(model, vectorizer, dtm_tf,
                                               n_top_words, lambda_)
    return [[entity_linker.link_entity(entity) for entity in topic]
            for topic in topic_terms]


In [8]:
linked_terms = link_topic_terms(entity_linker, lda_pipe.named_steps['model'],
                                lda_pipe.named_steps['vectorizer'], dtm_tf, 
                                n_top_words=10, lambda_=0.8)
linked_terms

[[('food', 'http://www.wikidata.org/entity/Q2095'),
  ('system', 'http://www.wikidata.org/entity/Q58778'),
  ('habitat', 'http://www.wikidata.org/entity/Q52105'),
  ('grassland', 'http://www.wikidata.org/entity/Q1006733'),
  ('production', 'http://www.wikidata.org/entity/Q739302'),
  ('agricultural', 'http://www.wikidata.org/entity/Q5356428'),
  ('land', 'http://www.wikidata.org/entity/Q11081619'),
  ('bird', 'http://www.wikidata.org/entity/Q5113'),
  ('change', 'http://www.wikidata.org/entity/Q1150070'),
  ('security', 'http://www.wikidata.org/entity/Q2526135')],
 [('plant', 'http://www.wikidata.org/entity/Q756'),
  ('expression', 'http://www.wikidata.org/entity/Q11024'),
  ('gene', 'http://www.wikidata.org/entity/Q7187'),
  ('adaptation', 'http://www.wikidata.org/entity/Q3331189'),
  ('root', 'http://www.wikidata.org/entity/Q111029'),
  ('shoot', 'http://www.wikidata.org/entity/Q220869'),
  ('site', 'http://www.wikidata.org/entity/Q35127'),
  ('diversification', 'http://www.wikidata.

## Obtaining each topic's graphs

In [10]:
import functools
import pdb

from dataclasses import dataclass

import networkx as nx


WIKIDATA_PROPS_EXPAND = ['P31', 'P279', 'P301', 'P361', 'P366',
                         'P527', 'P910', 'P921', 'P2578', 'P2579']


def empty_if_keyerror(function):
    """
    A decorator that wraps the passed in function and
    returns an empty string if a key error is raised.
    """
    @functools.wraps(function)
    def wrapper(*args, **kwargs):
        try:
            return function(*args, **kwargs)
        except KeyError:
            return ""
    return wrapper

def _build_uri(entity_id):
    return f"http://www.wikidata.org/entity/{entity_id}"

@empty_if_keyerror
def _get_aliases(entity_info, lang='en'):
    return [alias['value'] 
            for alias in entity_info['aliases'][lang]]

@empty_if_keyerror
def _get_desc(entity_info, lang='en'):
    return entity_info['descriptions'][lang]['value']

@empty_if_keyerror
def _get_labels(entity_info, lang='en'):
    return entity_info['labels'][lang]['value']


@dataclass
class WikidataNode():
    label: str
    uri: str
    desc: str
    alias: str
        
    def __hash__(self):
        return hash(self.uri)
    
    def to_dict(self):
        return {
            'alias':self.alias,
            'desc': self.desc,
            'label': self.label,
            'uri': self.uri
        }


class WikidataGraphBuilder():
    def __init__(self, max_hops=2, additional_props=None):
        self.max_hops = max_hops
        self.props_to_expand = WIKIDATA_PROPS_EXPAND
        if additional_props:
            self.props_to_expand += additional_props
    
    def build_graph(self, topic):
        G = nx.Graph()
        for term in topic:
            term_uri = term[1]
            if term_uri is not None:
                term_id = term_uri.split('/')[-1]
                self._add_wd_node_info(G, term_id, None, 0)
        return G
    
    def _add_wd_node_info(self, graph, term_id, prev_node, curr_hop):
        print(f"Visiting entity '{term_id}' - Curr hop: {curr_hop}")
        if curr_hop > self.max_hops or term_id == 'Q4167836':
            return
        
        # call wikidata API for uri
        endpoint = f"{WIKIDATA_BASE}/api.php?action=wbgetentities&ids={term_id}&languages=en&format=json"
        res = requests.get(endpoint)
        if res.status_code != 200:
            raise Error()
        
        content = json.loads(res.text)
        entity_info = content['entities'][term_id]
        
        if term_id not in graph.nodes:
            graph.add_node(term_id)
            #graph.nodes[term_id]['alias'] = _get_aliases(entity_info)
            graph.nodes[term_id]['desc'] = _get_desc(entity_info)
            graph.nodes[term_id]['label'] = _get_labels(entity_info)
            graph.nodes[term_id]['n'] = curr_hop

        if prev_node is not None and not graph.has_edge(prev_node, term_id):
            graph.add_edge(prev_node, term_id)
        
        for claim_key, claim_values in entity_info['claims'].items():
            if claim_key not in self.props_to_expand:
                continue
            
            for value in claim_values:
                snaktype = value['mainsnak']['snaktype']
                if snaktype in ['novalue', 'somevalue']:
                    continue
                
                new_node_id = value['mainsnak']['datavalue']['value']['id']
                self._add_wd_node_info(graph, new_node_id, term_id, curr_hop + 1)


In [11]:
graph_builder = WikidataGraphBuilder(max_hops=2)
topic_graphs = [graph_builder.build_graph(topic) for topic in linked_terms]

Visiting entity 'Q2095' - Curr hop: 0
Visiting entity 'Q2424752' - Curr hop: 1
Visiting entity 'Q28877' - Curr hop: 2
Visiting entity 'Q29028649' - Curr hop: 3
Visiting entity 'Q64513524' - Curr hop: 3
Visiting entity 'Q5672864' - Curr hop: 3
Visiting entity 'Q8134' - Curr hop: 3
Visiting entity 'Q337060' - Curr hop: 3
Visiting entity 'Q2897903' - Curr hop: 3
Visiting entity 'Q8205328' - Curr hop: 2
Visiting entity 'Q223557' - Curr hop: 3
Visiting entity 'Q16686448' - Curr hop: 3
Visiting entity 'Q26991679' - Curr hop: 3
Visiting entity 'Q15401930' - Curr hop: 2
Visiting entity 'Q488383' - Curr hop: 3
Visiting entity 'Q7189878' - Curr hop: 2
Visiting entity 'Q2424752' - Curr hop: 3
Visiting entity 'Q2897903' - Curr hop: 3
Visiting entity 'Q4167836' - Curr hop: 3
Visiting entity 'Q1194058' - Curr hop: 1
Visiting entity 'Q2424752' - Curr hop: 2
Visiting entity 'Q28877' - Curr hop: 3
Visiting entity 'Q8205328' - Curr hop: 3
Visiting entity 'Q15401930' - Curr hop: 3
Visiting entity 'Q71898

Visiting entity 'Q50377228' - Curr hop: 3
Visiting entity 'Q58778' - Curr hop: 0
Visiting entity 'Q6642953' - Curr hop: 1
Visiting entity 'Q58778' - Curr hop: 2
Visiting entity 'Q6642953' - Curr hop: 3
Visiting entity 'Q6671777' - Curr hop: 3
Visiting entity 'Q269699' - Curr hop: 3
Visiting entity 'Q123637' - Curr hop: 3
Visiting entity 'Q2167061' - Curr hop: 3
Visiting entity 'Q682496' - Curr hop: 3
Visiting entity 'Q7184903' - Curr hop: 3
Visiting entity 'Q3895768' - Curr hop: 3
Visiting entity 'Q4167836' - Curr hop: 2
Visiting entity 'Q6671777' - Curr hop: 1
Visiting entity 'Q35120' - Curr hop: 2
Visiting entity 'Q23958852' - Curr hop: 3
Visiting entity 'Q151885' - Curr hop: 3
Visiting entity 'Q77468620' - Curr hop: 3
Visiting entity 'Q6643007' - Curr hop: 2
Visiting entity 'Q6671777' - Curr hop: 3
Visiting entity 'Q4167836' - Curr hop: 3
Visiting entity 'Q269699' - Curr hop: 1
Visiting entity 'Q7036033' - Curr hop: 2
Visiting entity 'Q269699' - Curr hop: 3
Visiting entity 'Q4167836

Visiting entity 'Q1496967' - Curr hop: 3
Visiting entity 'Q6327738' - Curr hop: 3
Visiting entity 'Q43619' - Curr hop: 3
Visiting entity 'Q101998' - Curr hop: 1
Visiting entity 'Q107425' - Curr hop: 2
Visiting entity 'Q1496967' - Curr hop: 3
Visiting entity 'Q7143080' - Curr hop: 3
Visiting entity 'Q1529874' - Curr hop: 3
Visiting entity 'Q271669' - Curr hop: 3
Visiting entity 'Q30336093' - Curr hop: 2
Visiting entity 'Q58778' - Curr hop: 3
Visiting entity 'Q815297' - Curr hop: 3
Visiting entity 'Q28845870' - Curr hop: 3
Visiting entity 'Q7145637' - Curr hop: 3
Visiting entity 'Q96116695' - Curr hop: 3
Visiting entity 'Q7031839' - Curr hop: 2
Visiting entity 'Q101998' - Curr hop: 3
Visiting entity 'Q4167836' - Curr hop: 3
Visiting entity 'Q2083910' - Curr hop: 1
Visiting entity 'Q7725551' - Curr hop: 2
Visiting entity 'Q2083910' - Curr hop: 3
Visiting entity 'Q4167836' - Curr hop: 3
Visiting entity 'Q101998' - Curr hop: 2
Visiting entity 'Q107425' - Curr hop: 3
Visiting entity 'Q303360

Visiting entity 'Q2221906' - Curr hop: 3
Visiting entity 'Q35145263' - Curr hop: 3
Visiting entity 'Q15324' - Curr hop: 3
Visiting entity 'Q7144528' - Curr hop: 3
Visiting entity 'Q715269' - Curr hop: 3
Visiting entity 'Q83169' - Curr hop: 3
Visiting entity 'Q184395' - Curr hop: 3
Visiting entity 'Q11081619' - Curr hop: 2
Visiting entity 'Q205895' - Curr hop: 3
Visiting entity 'Q1349417' - Curr hop: 3
Visiting entity 'Q690092' - Curr hop: 3
Visiting entity 'Q786270' - Curr hop: 3
Visiting entity 'Q878077' - Curr hop: 2
Visiting entity 'Q35666' - Curr hop: 3
Visiting entity 'Q7484139' - Curr hop: 3
Visiting entity 'Q690092' - Curr hop: 3
Visiting entity 'Q41228' - Curr hop: 3
Visiting entity 'Q786270' - Curr hop: 1
Visiting entity 'Q17651246' - Curr hop: 2
Visiting entity 'Q151885' - Curr hop: 3
Visiting entity 'Q786270' - Curr hop: 3
Visiting entity 'Q690092' - Curr hop: 3
Visiting entity 'Q399984' - Curr hop: 3
Visiting entity 'Q399984' - Curr hop: 2
Visiting entity 'Q8508474' - Curr 

Visiting entity 'Q29000916' - Curr hop: 3
Visiting entity 'Q189447' - Curr hop: 3
Visiting entity 'Q4738155' - Curr hop: 3
Visiting entity 'Q2780922' - Curr hop: 1
Visiting entity 'Q9015082' - Curr hop: 2
Visiting entity 'Q4167836' - Curr hop: 3
Visiting entity 'Q2780922' - Curr hop: 3
Visiting entity 'Q851598' - Curr hop: 2
Visiting entity 'Q9424367' - Curr hop: 3
Visiting entity 'Q2526135' - Curr hop: 3
Visiting entity 'Q96264607' - Curr hop: 3
Visiting entity 'Q813912' - Curr hop: 1
Visiting entity 'Q151885' - Curr hop: 2
Visiting entity 'Q2145290' - Curr hop: 3
Visiting entity 'Q7184903' - Curr hop: 3
Visiting entity 'Q1347367' - Curr hop: 3
Visiting entity 'Q5550686' - Curr hop: 3
Visiting entity 'Q23958852' - Curr hop: 3
Visiting entity 'Q5891' - Curr hop: 2
Visiting entity 'Q11862829' - Curr hop: 3
Visiting entity 'Q4671286' - Curr hop: 3
Visiting entity 'Q1983674' - Curr hop: 3
Visiting entity 'Q44325' - Curr hop: 3
Visiting entity 'Q35277' - Curr hop: 3
Visiting entity 'Q9471'

...

Visiting entity 'Q420' - Curr hop: 3
Visiting entity 'Q4925926' - Curr hop: 3
Visiting entity 'Q420' - Curr hop: 3
Visiting entity 'Q7020' - Curr hop: 3
Visiting entity 'Q28598684' - Curr hop: 3
Visiting entity 'Q11862829' - Curr hop: 3
Visiting entity 'Q756' - Curr hop: 2
Visiting entity 'Q1457109' - Curr hop: 3
Visiting entity 'Q16521' - Curr hop: 3
Visiting entity 'Q41500' - Curr hop: 3
Visiting entity 'Q5935823' - Curr hop: 3
Visiting entity 'Q134267' - Curr hop: 3
Visiting entity 'Q660498' - Curr hop: 3
Visiting entity 'Q7201518' - Curr hop: 3
Visiting entity 'Q441' - Curr hop: 3
Visiting entity 'Q7239' - Curr hop: 3
Visiting entity 'Q16335078' - Curr hop: 2
Visiting entity 'Q7020' - Curr hop: 3
Visiting entity 'Q1056194' - Curr hop: 3
Visiting entity 'Q7201518' - Curr hop: 3
Visiting entity 'Q47263' - Curr hop: 2
Visiting entity 'Q165963' - Curr hop: 3
Visiting entity 'Q5058355' - Curr hop: 3
Visiting entity 'Q7201518' - Curr hop: 3
Visiting entity 'Q441' - Curr hop: 1
Visiting e

Visiting entity 'Q1310239' - Curr hop: 3
Visiting entity 'Q16686022' - Curr hop: 3
Visiting entity 'Q420' - Curr hop: 2
Visiting entity 'Q7991' - Curr hop: 3
Visiting entity 'Q1457402' - Curr hop: 3
Visiting entity 'Q2465832' - Curr hop: 3
Visiting entity 'Q11862829' - Curr hop: 3
Visiting entity 'Q3' - Curr hop: 3
Visiting entity 'Q7239' - Curr hop: 3
Visiting entity 'Q183252' - Curr hop: 3
Visiting entity 'Q7150' - Curr hop: 3
Visiting entity 'Q441' - Curr hop: 3
Visiting entity 'Q431' - Curr hop: 3
Visiting entity 'Q4785356' - Curr hop: 3
Visiting entity 'Q514' - Curr hop: 3
Visiting entity 'Q7175' - Curr hop: 3
Visiting entity 'Q19924411' - Curr hop: 2
Visiting entity 'Q8957823' - Curr hop: 3
Visiting entity 'Q11862829' - Curr hop: 3
Visiting entity 'Q4671286' - Curr hop: 3
Visiting entity 'Q2465832' - Curr hop: 3
Visiting entity 'Q4421' - Curr hop: 3
Visiting entity 'Q720362' - Curr hop: 3
Visiting entity 'Q36963' - Curr hop: 3
Visiting entity 'Q7239' - Curr hop: 3
Visiting entity

Visiting entity 'Q5' - Curr hop: 1
Visiting entity 'Q23852' - Curr hop: 2
Visiting entity 'Q6539712' - Curr hop: 3
Visiting entity 'Q170494' - Curr hop: 3
Visiting entity 'Q66599465' - Curr hop: 3
Visiting entity 'Q5' - Curr hop: 3
Visiting entity 'Q3409626' - Curr hop: 3
Visiting entity 'Q25459754' - Curr hop: 3
Visiting entity 'Q6027402' - Curr hop: 3
Visiting entity 'Q2261656' - Curr hop: 3
Visiting entity 'Q193818' - Curr hop: 3
Visiting entity 'Q9633' - Curr hop: 3
Visiting entity 'Q11392181' - Curr hop: 3
Visiting entity 'Q7891' - Curr hop: 3
Visiting entity 'Q42165' - Curr hop: 3
Visiting entity 'Q726543' - Curr hop: 3
Visiting entity 'Q9649' - Curr hop: 3
Visiting entity 'Q181100' - Curr hop: 3
Visiting entity 'Q11078' - Curr hop: 3
Visiting entity 'Q11068' - Curr hop: 3
Visiting entity 'Q712604' - Curr hop: 3
Visiting entity 'Q1059' - Curr hop: 3
Visiting entity 'Q483213' - Curr hop: 3
Visiting entity 'Q7895' - Curr hop: 3
Visiting entity 'Q3999' - Curr hop: 3
Visiting entity 

Visiting entity 'Q123619' - Curr hop: 3
Visiting entity 'Q206229' - Curr hop: 3
Visiting entity 'Q424689' - Curr hop: 3
Visiting entity 'Q50365941' - Curr hop: 3
Visiting entity 'Q7213592' - Curr hop: 3
Visiting entity 'Q28745' - Curr hop: 3
Visiting entity 'Q85209825' - Curr hop: 3
Visiting entity 'Q47154513' - Curr hop: 3
Visiting entity 'Q14330639' - Curr hop: 3
Visiting entity 'Q15311574' - Curr hop: 3
Visiting entity 'Q14330807' - Curr hop: 3
Visiting entity 'Q21763256' - Curr hop: 3
Visiting entity 'Q21105696' - Curr hop: 3
Visiting entity 'Q21136004' - Curr hop: 3
Visiting entity 'Q22325439' - Curr hop: 3
Visiting entity 'Q42244' - Curr hop: 3
Visiting entity 'Q21106243' - Curr hop: 3
Visiting entity 'Q22328923' - Curr hop: 3
Visiting entity 'Q22328924' - Curr hop: 3
Visiting entity 'Q22328925' - Curr hop: 3
Visiting entity 'Q417662' - Curr hop: 3
Visiting entity 'Q21119991' - Curr hop: 3
Visiting entity 'Q15334281' - Curr hop: 3
Visiting entity 'Q21123293' - Curr hop: 3
Visitin

Visiting entity 'Q9997097' - Curr hop: 3
Visiting entity 'Q29575627' - Curr hop: 3
Visiting entity 'Q9659' - Curr hop: 3
Visiting entity 'Q336228' - Curr hop: 3
Visiting entity 'Q9985' - Curr hop: 3
Visiting entity 'Q9705' - Curr hop: 3
Visiting entity 'Q9820' - Curr hop: 3
Visiting entity 'Q9884' - Curr hop: 3
Visiting entity 'Q16164' - Curr hop: 3
Visiting entity 'Q9907' - Curr hop: 3
Visiting entity 'Q10009' - Curr hop: 3
Visiting entity 'Q9739' - Curr hop: 3
Visiting entity 'Q9914' - Curr hop: 3
Visiting entity 'Q9893' - Curr hop: 3
Visiting entity 'Q9922' - Curr hop: 3
Visiting entity 'Q9927' - Curr hop: 3
Visiting entity 'Q9933' - Curr hop: 3
Visiting entity 'Q9937' - Curr hop: 3
Visiting entity 'Q9941' - Curr hop: 3
Visiting entity 'Q56161' - Curr hop: 3
Visiting entity 'Q394692' - Curr hop: 3
Visiting entity 'Q9946' - Curr hop: 3
Visiting entity 'Q9950' - Curr hop: 3
Visiting entity 'Q9852' - Curr hop: 3
Visiting entity 'Q9956' - Curr hop: 3
Visiting entity 'Q9813' - Curr hop: 

In [12]:
import networkx as nx

from bokeh.io import output_file, show
from bokeh.layouts import gridplot
from bokeh.models import (BoxZoomTool, Circle, HoverTool,
                          MultiLine, Plot, Range1d, ResetTool,)
from bokeh.palettes import Spectral4
from bokeh.plotting import from_networkx

def build_graph_plot(G, title=""):
    plot = Plot(plot_width=400, plot_height=400,
                x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1))
    plot.title.text = title

    SAME_CLUB_COLOR, DIFFERENT_CLUB_COLOR = "black", "red"
    node_attrs = {}

    for node in G.nodes(data=True):
        node_color = Spectral4[node[1]['n']]
        node_attrs[node[0]] = node_color

    nx.set_node_attributes(G, node_attrs, "node_color")

    node_hover_tool = HoverTool(tooltips=[("Label", "@label"), ("n", "@n")])
    plot.add_tools(node_hover_tool, BoxZoomTool(), ResetTool())

    graph_renderer = from_networkx(G, nx.spring_layout, scale=1, center=(0, 0))

    graph_renderer.node_renderer.glyph = Circle(size=15, fill_color="node_color")
    graph_renderer.edge_renderer.glyph = MultiLine(line_alpha=0.8, line_width=1)
    plot.renderers.append(graph_renderer)
    return plot


plots = [build_graph_plot(g, f"Topic {idx}") 
         for idx, g in enumerate(topic_graphs)]
grid = gridplot(plots, ncols=2)
show(grid)

## Getting the main connected subgraph

In [13]:
import networkx.algorithms as nxa

def get_largest_connected_subgraph(g):
    S = [g.subgraph(c).copy() for c in nxa.components.connected_components(g)]
    return max(S, key=len)


In [14]:
connected_topic_subgraphs = [get_largest_connected_subgraph(g) 
                             for g in topic_graphs]

In [15]:
plots = [build_graph_plot(g, f"Largest Connected subgraph for topic {idx}") 
         for idx, g in enumerate(connected_topic_subgraphs)]
grid = gridplot(plots, ncols=2)
show(grid)

## Obtaining the main component of each topic

In [19]:
def get_algorithm_results(topic_subgraphs, algorithm, stop_uris, top_n):
    topic_results = [algorithm(g) for g in topic_subgraphs]
    topic_results = [{key: val for key, val in result.items()
                      if topic_subgraphs[idx].nodes[key]['n'] != 0
                      and key not in stop_uris}
                      for idx, result in enumerate(topic_results)]
    topics_qids = [sorted(res, key=res.get, reverse=True)[:top_n] for res in topic_results]
    return [[(topic_subgraphs[idx].nodes[qid]['label'], qid) for qid in topic] 
            for idx, topic in enumerate(topics_qids)]


In [17]:
def try_algorithms(topic_subgraphs, algorithms, stop_uris, top_n=4):
    markdown = ""
    for (algorithm, name) in algorithms:
        print(f'Algorithm: {name}')
        results = get_algorithm_results(topic_subgraphs,
                                         algorithm, stop_uris,
                                         top_n)
        for idx, result in enumerate(results):
            print(f"Topic {idx}:", result)
            print()
        print()

        
algorithms = [
    (nxa.centrality.information_centrality, "Information centrality"),
    (nxa.centrality.eigenvector_centrality_numpy, "Eigenvector centrality"),
    (nxa.centrality.closeness_centrality, "Closeness centrality"),
    (nxa.centrality.betweenness_centrality, "Betweenness centrality"),
    (nxa.centrality.communicability_betweenness_centrality, "Communicability betweenness centrality")
]

try_algorithms(connected_topic_subgraphs,
               algorithms,
               ['Q4167836', 'Q11862829'])

Algorithm: Information centrality
Topic 0: [('action', 'Q4026292'), ('security engineering', 'Q365674'), ('activity', 'Q1914636'), ('eating', 'Q213449')]

Topic 1: [('interaction science', 'Q97008347'), ('communication studies', 'Q14623843'), ('communication medium', 'Q340169'), ('botany', 'Q441')]

Topic 2: [('agriculture', 'Q11451'), ('economic sector', 'Q3958441'), ('economic branch', 'Q268592'), ('cybernetics', 'Q123637')]

Topic 3: [('agriculture', 'Q11451'), ('cultivated plant', 'Q1362373'), ('food ingredient', 'Q25403900'), ('crop', 'Q235352')]

Topic 4: [('biological process', 'Q2996394'), ('group or class of chemical substances', 'Q17339814'), ('first-order metaclass', 'Q24017414'), ('biopolymer', 'Q422649')]

Topic 5: [('class', 'Q16889133'), ('group or class of proteins', 'Q84467700'), ('protein', 'Q8054'), ('first-order metaclass', 'Q24017414')]

Topic 6: [('biological process', 'Q2996394'), ('first-order metaclass', 'Q24017414'), ('group or class of chemical substances', '

The main funcionality from above will be implemented in a custom class that conforms to the sklearn API:

In [ ]:

class TopicLabeller():
    def __init__(self, graph_builder,
                 r=nxa.centrality.information_centrality,
                 num_labels_per_topic=1):
        self.graph_builder = graph_builder
        self.r = r
        self.num_labels = num_labels_per_topic
    
    def transform(self, X, y, **kwargs):
        return [self.get_topics_labels(topic) for topic in X]
    
    def get_topic_labels(self, topic_graph):
        topic_neighbourhood = self.graph_builder.build_graph(topic)
        subgraph = get_largest_connected_subgraph(topic_neighbourhood)
        metrics = r(subgraph) 
        # TODO remove seed concepts from metrics
        best_qids = max(metrics.items(), key=operator.itemgetter(1))
        return [subgraph[qid], metrics[qid] for qid in best_qids



## Add labels to LDA model

In [ ]:
final_results = get_algorithm_results(connected_topic_subgraphs,
                                      nxa.centrality.information_centrality,
                                      ['Q4167836', 'Q11862829'], top_n=1)

lda_model = lda_pipe.named_steps['model']
lda_model.topic_labels = final_results

In [ ]:
save_object(lda_pipe, os.path.join(NOTEBOOK_5_RESULTS_DIR, 'lda_pipe_with_labels.pkl'))